# jieba中文处理

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个工具去把完整的文本中分解成粒度更细的词。   

jieba就是这样一个非常好用的中文工具，是以分词起家的，但是功能比分词要强大很多。

## 1.基本分词函数与用法

jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)

jieba.cut 方法接受三个输入参数:
- 需要分词的字符串
- cut_all 参数用来控制是否采用全模式
- HMM 参数用来控制是否使用 HMM 模型

jieba.cut_for_search 方法接受两个参数
- 需要分词的字符串
- 是否使用 HMM 模型。

该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细

In [9]:
import jieba

# 全模式
seg_list = jieba.cut('我在学习自然语言处理', cut_all = True)
print(seg_list)
print('Full Mode:' + '/'.join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w2/qnnfb62x2g760j3nkh9q4ptr0000gn/T/jieba.cache


<generator object Tokenizer.cut at 0x10882d0f8>


Loading model cost 0.664 seconds.
Prefix dict has been built succesfully.


Full Mode:我/在/学习/自然/自然语言/语言/处理


In [10]:
# 精确模式
seg_list = jieba.cut('我在学习自然语言处理', cut_all = False)
print('Default Mode:' + '/'.join(seg_list))

Default Mode:我/在/学习/自然语言/处理


In [11]:
# 默认是精确模式
seg_list = jieba.cut('他毕业于上海交通大学，在百度深度学习研究院进行研究') 
print(','.join(seg_list))

他,毕业,于,上海交通大学,，,在,百度,深度,学习,研究院,进行,研究


In [12]:
# 搜索引擎模式
seg_list = jieba.cut_for_search('小明硕士毕业于中国科学院计算所，后在哈佛大学深造')
print(','.join(seg_list))

小明,硕士,毕业,于,中国,科学,学院,科学院,中国科学院,计算,计算所,，,后,在,哈佛,大学,哈佛大学,深造


### jie.cut以及jieba.cut_for_search直接返回list

In [13]:
result_lcut = jieba.lcut('小明硕士毕业于中国科学院计算所，后在哈佛大学深造')
print(result_lcut)
print()
print(' '.join(result_lcut))
print(' '.join(jieba.lcut_for_search('小明硕士毕业于中国科学院计算所，后在哈佛大学深造')))

['小明', '硕士', '毕业', '于', '中国科学院', '计算所', '，', '后', '在', '哈佛大学', '深造']

小明 硕士 毕业 于 中国科学院 计算所 ， 后 在 哈佛大学 深造
小明 硕士 毕业 于 中国 科学 学院 科学院 中国科学院 计算 计算所 ， 后 在 哈佛 大学 哈佛大学 深造


### 添加用户自定义词典

很多时候我们需要针对自己的场景进行分词，会有一些领域内的专有词汇。
- 1.可以用jieba.load_userdict(file_name)加载用户字典
- 2.少量的词汇可以自己用下面方法手动添加：
    - 用 add_word(word, freq=None, tag=None) 和 del_word(word) 在程序中动态修改词典
    - 用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。

In [14]:
# 如果是动态添加词频，HMM一定要设置为False
print('/'.join(jieba.cut('如果放在旧字典中将出错。', HMM=False)))

如果/放在/旧/字典/中将/出错/。


In [15]:
jieba.suggest_freq(('中','将'), True)

494

In [16]:
print('/'.join(jieba.cut('如果放到旧字典中将出错。', HMM=False)))

如果/放到/旧/字典/中/将/出错/。


### 关键词提取

#### 基于 TF-IDF 算法的关键词抽取
import jieba.analyse
- jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
    - sentence 为待提取的文本
    - topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
    - withWeight 为是否一并返回关键词权重值，默认值为 False
    - allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [17]:
import jieba.analyse as analyse

lines = open('./files/NBA.txt').read()

print(" ".join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

韦少 杜兰特 全明星 全明星赛 MVP 威少 正赛 科尔 投篮 勇士 球员 斯布鲁克 更衣柜 NBA 三连庄 张卫平 西部 指导 雷霆 明星队


In [18]:
for word_weight in analyse.extract_tags(lines, topK=10, withWeight=True, allowPOS=()):
    print(word_weight)

('韦少', 0.40006111722303145)
('杜兰特', 0.3557260029055118)
('全明星', 0.30987389050216535)
('全明星赛', 0.28603743738661414)
('MVP', 0.28239608274566924)
('威少', 0.2588630758501968)
('正赛', 0.14119804137283462)
('科尔', 0.11543555039681103)
('投篮', 0.10367501793307088)
('勇士', 0.1036619725106693)


In [ ]:
with open('./files/西游记.txt', 'r',encoding='gbk') as f:
    lines = ''
    while True:
        try:
            lines += f.readline()
        except:
            pass
    
    print(' '.join(analyse.extract_tags(lines, topK=20, withWeight=False, allowPOS=())))

#### 关于TF-IDF 算法的关键词抽取补充
- 关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径
    - 用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径
        - 自定义语料库示例见<a href="https://github.com/fxsjy/jieba/blob/master/extra_dict/idf.txt.big">这里</a>
        - 用法示例见<a href="https://github.com/fxsjy/jieba/blob/master/test/extract_tags_idfpath.py">这里</a>
    - 关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径
        - 用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径
        - 自定义语料库示例见<a href="https://github.com/fxsjy/jieba/blob/master/extra_dict/stop_words.txt">这里</a>
        - 用法示例见<a href="https://github.com/fxsjy/jieba/blob/master/test/extract_tags_stop_words.py">这里</a>
- 关键词一并返回关键词权重值示例
    - 用法示例见<a href="https://github.com/fxsjy/jieba/blob/master/test/extract_tags_with_weight.py">这里</a>

##### 自定义IDF语料库和停止词语料库添加用法示例

In [4]:
import jieba
import jieba.analyse 

file_name = './files/NBA.txt'
content = open(file_name, 'r').read()

jieba.analyse.set_stop_words("./extra_dict/stop_words.txt")
jieba.analyse.set_idf_path('./extra_dict/idf.txt.big')

tags = jieba.analyse.extract_tags(content, topK=20)

print(' '.join(tags))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w2/qnnfb62x2g760j3nkh9q4ptr0000gn/T/jieba.cache
Loading model cost 0.693 seconds.
Prefix dict has been built succesfully.


韦少 全明星 杜兰特 MVP 全明星赛 威少 正赛 球员 勇士 自己 一个 指导 科尔 投篮 但是 斯布鲁克 就是 成为 NBA 加入


##### 关键词一并返回关键词权重值示例

In [8]:
import jieba 
import jieba.analyse

file_name = './files/NBA.txt'
content = open(file_name, 'r').read()

tags = jieba.analyse.extract_tags(content, topK=20, withWeight=True, allowPOS=())

for tag in tags:
    print("tag: %s\t\t weight: %f" % (tag[0],tag[1]))

tag: 韦少		 weight: 0.400061
tag: 全明星		 weight: 0.309874
tag: 杜兰特		 weight: 0.305929
tag: MVP		 weight: 0.282396
tag: 全明星赛		 weight: 0.282396
tag: 威少		 weight: 0.258863
tag: 正赛		 weight: 0.141198
tag: 球员		 weight: 0.141198
tag: 勇士		 weight: 0.141198
tag: 自己		 weight: 0.141198
tag: 一个		 weight: 0.117665
tag: 指导		 weight: 0.117665
tag: 科尔		 weight: 0.115436
tag: 投篮		 weight: 0.103675
tag: 但是		 weight: 0.094132
tag: 斯布鲁克		 weight: 0.094132
tag: 就是		 weight: 0.094132
tag: 成为		 weight: 0.094132
tag: NBA		 weight: 0.070599
tag: 加入		 weight: 0.070599
